# DCJ Project Rewrite with Megan
Algorithm based on information from:
- A Unifying View of Genome Rearrangements by Bergeron et al.
- Heuristics for Genome Rearrangement Distance With Replicated Genes by Siqueira et al.

In [26]:
using Parameters

In [27]:
LINEAR_PATH = 1
CYCLIC_PATH = 2

2

In [55]:
mutable struct DCJ_edge
    label::String
    tail_connection::Union{DCJ_edge, Nothing}
    head_connection::Union{DCJ_edge, Nothing}
end

function get_edge_count(edge::DCJ_edge)
    if edge.tail_connection === nothing || edge.head_connection === nothing
        return 1
    end
    return 2
end

mutable struct DCJ_graph
    # Holds a root edge, of which all other edges are connected. By nature each DCJ_graph object only holds one cycle/path
    root_edge::DCJ_edge
    type::Int
end

mutable struct DCJ_genome
    # Holds a genome - a collection of either one or several DCJ_graphs. A genome can be either connected in one large path or split into several paths within the genome
    graphs::Vector{DCJ_graph}
end

function display_genome(genome::DCJ_genome)
    for graph in genome.graphs
        display_dcj_graph(graph)
        println("-\n")
    end
end

function display_dcj_graph(g::DCJ_graph)
    # Displays a full DCJ_graph
    # First, loop through graph and find first telomere(external node)
    # Start displaying from the first telomere found
    telo = nothing
    root = g.root_edge
    trail = root
    edges = Set{DCJ_edge}()
    if g.type == LINEAR_PATH
        println("Linear Graph")
    else
        println("Cyclic Graph")
    end
    while trail !== nothing
        push!(edges, trail)
        head = trail.head_connection
        tail = trail.tail_connection
        if head in edges
            trail = tail
        else
            trail = head
        end
        if trail in edges
            break
        end
    end
    for edge in edges
        if get_edge_count(edge) == 1 # Found telomere
            telo = edge
            break
        end
    end
    if telo !== nothing
        # Displaying a linear path.
        # Create a set of seen edges(probably temporary) to make sure we are going the right way
        seen = Set{DCJ_edge}()
        push!(seen, telo)
        print(telo.label)
        # Get next connected edge to the initial telomere
        trail = telo.tail_connection
        if trail === nothing
            trail = telo.head_connection
        end
        while trail !== nothing
            print(trail.label)
            push!(seen, trail)
            trailhead = trail.head_connection
            trailtail = trail.tail_connection
            if trailhead in seen
                trail = trailtail
            else
                trail = trailhead
            end
        end
    else
        # Drawing a cyclic graph. Last character of the printed string connects to first letter of the output
        seen = Set{DCJ_edge}()
        trail = root
        while !(trail in seen)
            push!(seen, trail)
            print(trail.label)
            head = trail.head_connection
            tail = trail.tail_connection
            trail = head
            if head in seen
                trail = tail
            end
        end
    end
end

Base.Meta.ParseError: ParseError:
# Error @ /Users/seawied/Master/redone_megan_project/structs.ipynb:100:4
    end
end
#  └ ── Expected `end`

In [29]:
edge1 = DCJ_edge("a", nothing, nothing)
edge2 = DCJ_edge("b", edge1, nothing)
edge1.head_connection = edge2
edge3 = DCJ_edge("c", edge2, nothing)
edge2.head_connection = edge3
graph = DCJ_graph(edge1, LINEAR_PATH)
display_dcj_graph(graph)

Linear Graph
cba

In [30]:
function convert_string_to_graph(input::String, type::Int)
    if length(input) < 2
        print("Too short input string")
    end
    root_edge = DCJ_edge(string(input[1]), nothing, nothing)
    prev_edge = root_edge
    for entry in SubString(input, 2, length(input))
        new_edge = DCJ_edge(string(entry), prev_edge, nothing)
        prev_edge.head_connection = new_edge
        prev_edge = new_edge
    end
    if type == CYCLIC_PATH
        root_edge.tail_connection = prev_edge
        prev_edge.head_connection = root_edge
    end
    return DCJ_graph(root_edge, type)
end

convert_string_to_graph (generic function with 1 method)

In [31]:
convert_to_graph_test = convert_string_to_graph("abcdefghijklmnop", CYCLIC_PATH)
display_dcj_graph(convert_to_graph_test)

Cyclic Graph
abcdefghijklmnop

In [61]:
# More discrete control over graph/genome creation.
# This function takes in a string with head and tail connections more clearly typed in.
# Format: aT,aHcT,cHdT,dH - This is a complete cycle. T and H are reserved characters for distinguishing the individual units of the string
# This allows the creation of an entire genome from one graph.
function assemble_genome(input::String)
    genome = DCJ_genome(Vector{DCJ_graph}())
    current_graph = nothing
    sliced = split(input, ",")
    current_label = ""

    end
end

assemble_genome (generic function with 1 method)

In [62]:
test = "aT,aHcT,cHdT,dH"
assemble_genome(test)

aT
aHcT
cHdT
dH


In [46]:
function assemble_lookup_table(g::DCJ_graph)
    # NO DUPLICATES!
    lookup = Dict{String, DCJ_edge}()
    trail = g.root_edge
    edges = Set{DCJ_edge}()
    while trail !== nothing
        get!(lookup, trail.label, trail)
        push!(edges, trail)
        head = trail.head_connection
        tail = trail.tail_connection
        if head in edges
            trail = tail
        else
            trail = head
        end
        if trail in edges
            break
        end
    end
    return lookup
end

assemble_lookup_table (generic function with 1 method)

In [54]:
# Lookup table test
test = convert_string_to_graph("abcdefghijklmnop", LINEAR_PATH)
table = assemble_lookup_table(test)
display_dcj_graph(test)
println("")
for (key, value) in table
    println("Key: $key, Value: $value")
end
print(table["f"])

Linear Graph
abcdefghijklmnop
Key: f, Value: DCJ_edge("f", DCJ_edge("e", DCJ_edge("d", DCJ_edge("c", DCJ_edge("b", DCJ_edge("a", nothing, DCJ_edge(#= circular reference @-2 =#)), DCJ_edge(#= circular reference @-2 =#)), DCJ_edge(#= circular reference @-2 =#)), DCJ_edge(#= circular reference @-2 =#)), DCJ_edge(#= circular reference @-2 =#)), DCJ_edge("g", DCJ_edge(#= circular reference @-2 =#), DCJ_edge("h", DCJ_edge(#= circular reference @-2 =#), DCJ_edge("i", DCJ_edge(#= circular reference @-2 =#), DCJ_edge("j", DCJ_edge(#= circular reference @-2 =#), DCJ_edge("k", DCJ_edge(#= circular reference @-2 =#), DCJ_edge("l", DCJ_edge(#= circular reference @-2 =#), DCJ_edge("m", DCJ_edge(#= circular reference @-2 =#), DCJ_edge("n", DCJ_edge(#= circular reference @-2 =#), DCJ_edge("o", DCJ_edge(#= circular reference @-2 =#), DCJ_edge("p", DCJ_edge(#= circular reference @-2 =#), nothing)))))))))))
Key: c, Value: DCJ_edge("c", DCJ_edge("b", DCJ_edge("a", nothing, DCJ_edge(#= circular reference @

In [21]:
SWAP_OPCODE = 1
SPLIT_OPCODE = 2
COMBINE_OPCODE = 3

3

In [22]:
function apply_dcj_op(opcode::Int, graph::DCJ_graph)
    if opcode == SWAP_OPCODE
        swap_op(graph)
    else if opcode == SPLIT_OPCODE
        split_op(graph)
    elseif opcode == COMBINE_OPCODE
        combine_op(graph)
    end
end

function swap_op(graph::DCJ_graph)
end

function split_op(graph::DCJ_graph)
end

function combine_op(graph::DCJ_graph)
end

Base.Meta.ParseError: ParseError:
# Error @ /Users/seawied/Master/redone_megan_project/structs.ipynb:4:5
        swap_op(graph)
    else if opcode == SPLIT_OPCODE
#   └─────┘ ── use `elseif` instead of `else if`